# Google_trend_data_select_by_deferred

## INFO
- \[Encoding\] : `utf-8`
- \[Author\] : `yen-nan ho`
- \[Contact\] : `aaron1aaron2@gmail.com`
- \[GitHub\] : `https://github.com/aaron1aaron2`
- \[Create Date\] :  `20210910`



## 輔助 function

In [ ]:
import pandas as pd
import os

def combine_google_trend_data(data_path):
    """合併個關鍵字手動下載的結果"""
    file_ls = [i for i in os.listdir(data_path) if i.find('.csv')!=-1]

    result = pd.DataFrame()
    for file in file_ls:
        popularity = pd.read_csv(os.path.join(data_path, file), skiprows=3, header=None)
        popularity.columns = ['date','search_rate']
        popularity['keyword'] = keyword

        result = result.append(popularity)

    result = pd.concat([result, result['date'].str.extract('(?P<year>\d+)-(?P<month>\d+)-\d+')], axis=1)

    return result

## 1. 清整資料

In [ ]:
import pandas as pd

# 將 google trand 的原始資料集合起來
googletrend_df = combine_google_trend_data("data/raw/google_trend/spots")
googletrend_df.to_csv("data/proccess/2_google_trend/googletrend_keyword_combine.v1.py", index=False)